# 1.KNN estimator with a kernel funtion (20 pts)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(2023)

## 1.1Generate data points

In [ ]:
x_trian = np.concatenate([np.random.normal(-20, 10, 20), np.random.normal(10, 5, 10)])
x_trian =x_trian.reshape((-1,1))
x_test = np.linspace(np.min(x_trian),np.max(x_trian),100)
x_test = x_test.reshape((-1,1))


In [ ]:
#  sampling  many dpoins for approximating the true distribution
# this is a multimodal density
sns.kdeplot(np.concatenate([np.random.normal(-20, 10, 100), np.random.normal(10, 5, 100)]))
plt.show()

## 1.2 Complete the code and draw the pictures
Completing KNN estimator wtih a kernel funtion. 
The kernel funtion is a Gaussian kernel, defined by $$K(u)=\frac{1}{\sqrt{2\pi}}exp(-\frac{u^2}{\tau}),$$
where $\tau$ is the length-scale. 

**Task:** You should complete the code for the KNN estimator with a Gaussian kernel function and plot the estimated density with three sets of parameters, as shown below
- k=3,$\tau=2$
- k=3,$\tau=0.2$
- k=5,$\tau=2$

**Hint:** We have already generated a set of test points and saved them in the variable x_test. Your task is to plot the estimated density function $\hat{p}(x)$ based on these points. The code framework has been provided, and you can either write it from scratch or fill in the missing parts in the framework.

In [ ]:
##########################
#Define your KNN estimator 
#########################






In [ ]:
k=3
tau=2
##################
#Fill in the blanks by your code or using following template
# kke = KNNKernelEstimator(k,tau)
# kke.fit(x_trian)
# predict_x = kke.predict(x_test)
###################

plt.figure(figsize=(10,6))
plt.plot(x_test,predict_x)
plt.title("k:"+str(k)+r"   $\tau$:"+str(tau),fontsize=20)
plt.show()

k=3
tau=0.2
##################
#Fill in the blanks by your code
###################
plt.figure(figsize=(10,6))
plt.plot(x_test,predict_x)
plt.title("k:"+str(k)+r"   $\tau$:"+str(tau),fontsize=20)
plt.show()

k=5
tau=2
##################
#Fill in the blanks by your code
###################
plt.figure(figsize=(10,6))
plt.plot(x_test,predict_x)
plt.title("k:"+str(k)+r"   $\tau$:"+str(tau),fontsize=20)
plt.show()

# 2.Deep Learning for classification (30 pts)

For this task, you are required to design and train a deep neural network to perform a classification task on a provided dataset. The dataset can be found at the following link: http://pan.shanghaitech.edu.cn/cloudservice/outerLink/decode?c3Vnb24xNjgyNzcwODk4OTU5c3Vnb24=

The dataset consists of a training set and test set. The training set should be used to train your model, and the test set should be used to evaluate the performance of your model.

**Your goal is to achieve at least 70% accuracy on the test set using your trained model. One point is deducted for every point the accuracy decreases**， such as 66.2% will lose 4 points.

If your computer does not have the necessary resources to train a deep neural network, you may use the computing resources of a school computing cluster, Kaggle, or Google Colab.

Hint:The use of pre-training models is prohibited (direct zero points) and custom neural networks are encouraged.



In [ ]:
import os
import numpy as np
# !pip install opencv-python  # the command for installing opencv,i.e, cv2
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
import warnings
import  torch.nn.functional as F

warnings.filterwarnings("ignore")


In [ ]:
# load the data set
def readfile(path, label):
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [ ]:
workspace_dir = './food-11'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
test_x, test_y = readfile(os.path.join(workspace_dir, "testing"), True)
print("Size of test data = {}".format(len(test_x)))

In [ ]:

train_transform = transforms.Compose([
            ######################
            #Fill in the blanks by your code
            #########################
])
test_transform = transforms.Compose([
    ######################
    #Fill in the blanks by your code
    #########################
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [ ]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = ImgDataset(test_x, test_y, test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# 2.2 Construct deep learning model

In [ ]:
################################
#Define your nerual network
##################################
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        ######################
        #   Use the pytorch API to build the neural network
        #########################


    def forward(self, x):
        

     

## 2.3 trianing your model
There are some tips which I hope can help you complete the task:
- If the loss function goes down too slowly, you can make the step size larger. This trick is better combined with an adaptive learning rate regulator
- If the model is overfitting, you can add a dropout layer in your model. 
-  [Data augmentation](https://pytorch.org/vision/stable/transforms.html) is also a good way to increase model generalization. We recommend that you do this.


In [ ]:
model = Classifier().cuda()
loss = 
optimizer = 
num_epoch = 

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model.train()
    for i, data in enumerate(train_loader):

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_set.__len__(), train_loss/train_set.__len__()))

# Test your model
use your trained model to test the test set and print the accuracy.

In [ ]:

model.eval()
test_acc = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = 
        test_acc +=np.sum(np.argmax(test_pred.cpu().data.numpy(), axis=1) == data[1].numpy())

test_acc = test_acc/test_set.__len__()
print("Test Acc: "+str(test_acc))